In [1]:
import pandas as pd
from datetime import datetime
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler

In [2]:
ratings = pd.read_csv('u.data', names = ['user_id', 'item_id', 'rating', 'timestamp'], sep = '\t', header=None)
ratings.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [5]:
# ratings.info()

In [4]:
movies = pd.read_csv('u.item', names = ['movie_id', 'movie_title', 'release_date', 'video_release_year', 'IMDb_URL', 'unknown',
                                        'Action', 'Adventure', 'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary', 
                                        'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
                                        'Thriller', 'War', 'Western'], sep = '|', header=None, encoding="ISO-8859-1")
movies.head()

,movie_id,movie_title,release_date,video_release_year,IMDb_URL,unknown,Action,Adventure,Animation,Childrens,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [6]:
# movies.info()

In [7]:
# Найдем пользователя, поставившего больше всего оценок:
ratings.user_id.value_counts().head() # >> user_id = 405

405    737
655    685
13     636
450    540
276    518
Name: user_id, dtype: int64

In [8]:
# Отберем фильмы, которые оценил данный пользователь:
movies_user405 = ratings[ratings.user_id == 405]
movies_user405.head()

,user_id,item_id,rating,timestamp
12276,405,56,4,885544911
12383,405,592,1,885548670
12430,405,1582,1,885548670
12449,405,171,1,885549544
12460,405,580,1,885547447


In [9]:
movies_user405.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 737 entries, 12276 to 99756
Data columns (total 4 columns):
user_id      737 non-null int64
item_id      737 non-null int64
rating       737 non-null int64
timestamp    737 non-null int64
dtypes: int64(4)
memory usage: 28.8 KB


In [10]:
len(movies_user405['item_id'].unique()) # >> все фильмы - уникальные

737

In [11]:
movies[movies.movie_id == 592].head() # для каждого фильма указаны несколько жанров

,movie_id,movie_title,release_date,video_release_year,IMDb_URL,unknown,Action,Adventure,Animation,Childrens,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
591,592,True Crime (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?True%20Crime%...,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0


In [12]:
movies_user405.head()

,user_id,item_id,rating,timestamp
12276,405,56,4,885544911
12383,405,592,1,885548670
12430,405,1582,1,885548670
12449,405,171,1,885549544
12460,405,580,1,885547447


In [13]:
# Для построения модели будем использовать сл. признаки:
# год релиза, жанры, общее кол-во оценок для фильмов (от всех пользователей), средние оценки фильмов (от всех пользователей) 

In [14]:
# Добавим столбец с годом релиза:
movies['release_year'] = pd.DatetimeIndex(movies['release_date']).year.values.astype('int32')
movies.head()

,movie_id,movie_title,release_date,video_release_year,IMDb_URL,unknown,Action,Adventure,Animation,Childrens,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,release_year
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,1995
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,1,0,0,1995
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1995
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1995
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1995


In [39]:
# movies.info()

In [15]:
# Получим для каждого фильма количество оценок от всех пользователей и средние оценки:
item_ratings = ratings.pivot_table(index = 'item_id', values = 'rating', aggfunc = ['count', 'mean']).reset_index()
item_ratings.head(10)

,item_id,count,mean
,,rating,rating
0,1,452,3.878319
1,2,131,3.206107
2,3,90,3.033333
3,4,209,3.550239
4,5,86,3.302326
5,6,26,3.576923
6,7,392,3.798469
7,8,219,3.995434
8,9,299,3.896321


In [16]:
# Переименуем столбцы:
item_ratings.columns = ['item_id','number_of_ratings', 'average_rating']
item_ratings.head()

,item_id,number_of_ratings,average_rating
0,1,452,3.878319
1,2,131,3.206107
2,3,90,3.033333
3,4,209,3.550239
4,5,86,3.302326


In [17]:
len(item_ratings['item_id'].unique())

1682

In [18]:
# Создадим датафрейм для модели машинного обучения:

In [19]:
# Добавим столбцы с необходимыми признаками:
# кол-во и средние значения оценок для каждого фильма:
data = pd.merge(left=movies_user405, right=item_ratings, left_on='item_id', right_on='item_id')
data.head()

,user_id,item_id,rating,timestamp,number_of_ratings,average_rating
0,405,56,4,885544911,394,4.060914
1,405,592,1,885548670,9,3.333333
2,405,1582,1,885548670,1,1.000000
3,405,171,1,885549544,65,3.876923
4,405,580,1,885547447,32,3.375000


In [20]:
# год релиза и жанры:
df = pd.merge(left=data, right=movies, left_on='item_id', right_on='movie_id')
df.head()

,user_id,item_id,rating,timestamp,number_of_ratings,average_rating,movie_id,movie_title,release_date,video_release_year,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,release_year
0,405,56,4,885544911,394,4.060914,56,Pulp Fiction (1994),01-Jan-1994,NaN,...,0,0,0,0,0,0,0,0,0,1994
1,405,592,1,885548670,9,3.333333,592,True Crime (1995),01-Jan-1995,NaN,...,0,0,0,1,0,0,1,0,0,1995
2,405,1582,1,885548670,1,1.000000,1582,T-Men (1947),01-Jan-1947,NaN,...,1,0,0,0,0,0,0,0,0,1947
3,405,171,1,885549544,65,3.876923,171,Delicatessen (1991),01-Jan-1991,NaN,...,0,0,0,0,0,1,0,0,0,1991
4,405,580,1,885547447,32,3.375000,580,"Englishman Who Went Up a Hill, But Came Down a...",01-Jan-1995,NaN,...,0,0,0,0,1,0,0,0,0,1995


In [21]:
# df.columns

In [22]:
df = df[['movie_id', 'release_year', 'number_of_ratings','average_rating', 
       'unknown', 'Action', 'Adventure','Animation', 'Childrens', 'Comedy', 'Crime', 
       'Documentary', 'Drama','Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 
       'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western', 'rating']]

In [23]:
# все данные - числовые, пропусков нет
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 737 entries, 0 to 736
Data columns (total 24 columns):
movie_id             737 non-null int64
release_year         737 non-null int32
number_of_ratings    737 non-null int64
average_rating       737 non-null float64
unknown              737 non-null int64
Action               737 non-null int64
Adventure            737 non-null int64
Animation            737 non-null int64
Childrens            737 non-null int64
Comedy               737 non-null int64
Crime                737 non-null int64
Documentary          737 non-null int64
Drama                737 non-null int64
Fantasy              737 non-null int64
Film-Noir            737 non-null int64
Horror               737 non-null int64
Musical              737 non-null int64
Mystery              737 non-null int64
Romance              737 non-null int64
Sci-Fi               737 non-null int64
Thriller             737 non-null int64
War                  737 non-null int64
Western      

In [24]:
# подготовим данные
X, y = df.drop('rating', axis=1).values, df['rating'].values # np-матрицы

In [25]:
train_part_size = int(0.8 * X.shape[0]) # размер обучающей выборки
X_train, X_valid = X[:train_part_size, :], X[train_part_size:, :] # обучающая и отложенная части
y_train, y_valid = y[:train_part_size], y[train_part_size:]

In [26]:
# масштабируем данные
scaler = StandardScaler() # при обучении этот объект посчитает среднее и станд. отклонение по каждому признаку (обучится)
X_train_scaled = scaler.fit_transform(X_train) # масштабированная обучающая выборка (fit - обучение scaler, transform - возвращение масштабированной выборки)
X_valid_scaled = scaler.transform(X_valid) # масштабированная отложенная выборка (масштабирование с помощью посчитанных средних и станд. отклонений)

In [27]:
# LinearRegression
model = LinearRegression()

In [28]:
# обучение модели на масштабированных данных
model.fit(X_train_scaled, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [29]:
# формируем прогноз
y_pred = model.predict(X_valid_scaled)

In [30]:
# стандартное отклонение (среднеквадратическое отклонение) между ответами на отложенной выборке и прогнозами на масштабированной отложенной выборке
print('RMSE=%.2f' % mean_squared_error(y_valid, y_pred, squared=False))

RMSE=1.19


In [31]:
# коэффициент детерминации (какой процент наблюдений описывает данная модель?)
print('R^2 score=%.2f' % r2_score(y_valid, y_pred))

R^2 score=0.20


In [32]:
# средняя абсолютная погрешность регрессии (прогноза)
print('MAE=%.2f' % mean_absolute_error(y_valid, y_pred))

MAE=0.89


In [33]:
# DecisionTreeRegressor (для сравнения)
model = DecisionTreeRegressor()

In [34]:
# обучение модели на масштабированных данных
model.fit(X_train_scaled, y_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [35]:
# формируем прогноз
y_pred = model.predict(X_valid_scaled)

In [36]:
# стандартное отклонение (среднеквадратическое отклонение) между ответами на отложенной выборке и прогнозами на масштабированной отложенной выборке
print('RMSE=%.2f' % mean_squared_error(y_valid, y_pred, squared=False))

RMSE=1.55


In [37]:
# коэффициент детерминации (какой процент наблюдений описывает данная модель?)
print('R^2 score=%.2f' % r2_score(y_valid, y_pred))

R^2 score=-0.34


In [38]:
# средняя абсолютная погрешность регрессии (прогноза)
print('MAE=%.2f' % mean_absolute_error(y_valid, y_pred))

MAE=0.92


In [39]:
# В данном случае DecisionTreeRegressor по сравнению с LinearRegression дает большее значение всех рассчитанных коэффициентов